# UCU - Proyecto Webscraping
Proyecto para la Licenciatura en Datos y Negocios de la Universidad Católica del Uruguay

Importamos la librería requests para hacer las solicitudes HTTP
Importamos la librería BeautifulSoup para analizar el HTML de la página

In [15]:
import requests
from bs4 import BeautifulSoup
import json
import time
from collections import defaultdict

Inicialización de parámetros y estructura de resultados

In [ ]:
ciudades_objetivo = ["Montevideo", "Maldonado", "Canelones"]

# Inicializar resultados
resultado = {ciudad: [] for ciudad in ciudades_objetivo}

Aqui hacemos la paginación hasta completar 10 propiedades por ciudad

In [ ]:

page = 1
# Seguimos hasta que cada ciudad tenga 10 propiedades
while any(len(resultado[ciudad]) < 10 for ciudad in ciudades_objetivo):
    url = f"https://www.infocasas.com.uy/alquiler/casas-y-apartamentos/uruguay/pagina-{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

Realizamos la extracción de campos por página

In [ ]:

    # Ubicaciones
    location = soup.find_all('strong', class_='lc-location body body-1 body-bold medium')
    # Precios
    price = soup.find_all('p', class_='main-price')
    # Links
    links = soup.find_all('a', class_='lc-data', href=True)

    if not location or not price or not links:  # si ya no hay más propiedades
        break

Aqui realizamos la  extracción por propiedad

In [ ]:

    # Iterar por todas las propiedades de la página
    for loc, p, link_tag in zip(location, price, links):
        ciudad_prop = loc.get_text().split(",")[-1].strip()
        precio = p.get_text(strip=True)
        link = "https://www.infocasas.com.uy" + link_tag['href']

Aqui realizamos el filtrado y almacenamiento por ciudad (límite de 10)

In [ ]:

        # Agregar solo si es una de las ciudades que buscamos y no llegamos a 10
        if ciudad_prop in ciudades_objetivo and len(resultado[ciudad_prop]) < 10:
            resultado[ciudad_prop].append({
                "ciudad": ciudad_prop,
                "precio": precio,
                "link": link,
            })

    page += 1

Guardamos en JSON

In [16]:

# Guardar en JSON
with open("data.json", "w", encoding="utf-8") as outfile:
    json.dump(resultado, outfile, indent=4, ensure_ascii=False)